In [ ]:
# import os

# i2m = list(zip(range(1,13), ['Gener','Febrer','Marc','Abril','Maig','Juny','Juliol','Agost','Setembre','Octubre','Novembre','Desembre']))
# for year in [2023, 2022, 2021, 2020]:
#     for month, month_name in i2m:
#         os.system(f"wget 'https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
#         os.system(f"7z x '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")
#         os.system(f"rm '{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z'")

In [ ]:
!pip  install py7zr


In [ ]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import py7zr

# Create a target directory if it doesn't exist
target_directory = "bicing_data"
os.makedirs(target_directory, exist_ok=True)

i2m = list(zip(range(1, 13), ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig',
           'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']))


def download_and_extract(year, month, month_name):
    url = f"https://opendata-ajuntament.barcelona.cat/resources/bcn/BicingBCN/{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"
    local_filename = f"{year}_{month:02d}_{month_name}_BicingNou_ESTACIONS.7z"
    try:
        # Download the file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for request errors
        with open(local_filename, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        # Extract the file to the target directory
        with py7zr.SevenZipFile(local_filename, mode='r') as archive:
            archive.extractall(path=target_directory)

        # Remove the downloaded .7z file
        os.remove(local_filename)
    except Exception as e:
        print(f"Failed to download or extract {local_filename}: {e}")


# Use ThreadPoolExecutor for parallel downloading
with ThreadPoolExecutor(max_workers=4) as executor:
    futures = [executor.submit(download_and_extract, year, month, month_name)
               for year in [2023, 2022, 2021, 2020]
               for month, month_name in i2m]

    for future in as_completed(futures):
        if future.exception() is not None:
            print(f"Generated an exception: {future.exception()}")

print("All downloads and extractions are complete.")